In [10]:
pip install C:\Users\kapil\OneDrive\Desktop\NLP_Uzair\Project\Reddit_Sarcasm_Detection-main\Reddit_Sarcasm_Detection-main\src\data\raw\fasttext-0.9.2-cp39-cp39-win_amd64.whl

Processing c:\users\kapil\onedrive\desktop\nlp_uzair\project\reddit_sarcasm_detection-main\reddit_sarcasm_detection-main\src\data\raw\fasttext-0.9.2-cp39-cp39-win_amd64.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install fasttext

In [2]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import re,nltk,swifter
import csv
import fasttext
import matplotlib.pyplot as plt
import seaborn as sn
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import tree
from zeugma.embeddings import EmbeddingTransformer
## 1. Load dataset

## 1. Load dataset

In [3]:
#For reading data, we have to put path of the csv data file.
df = pd.read_csv(r'C:\Users\kapil\OneDrive\Desktop\NLP_Uzair\Project\Reddit_Sarcasm_Detection-main\Reddit_Sarcasm_Detection-main\src\data\train-balanced-sarcasm.csv')
df = df.fillna('')
df = df[['label','comment','author','score','created_utc','parent_comment']]
df.head()

,label,comment,author,score,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,2,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,-4,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,3,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,-8,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,6,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [4]:
df1 = df[['label','comment']]

# Cleaning comments

In [5]:
#Set of stop words excluding some of the words which show negation since its necessary for sarcasm detection:
stops = set(stopwords.words('english')) - {'no','not','nor','against','above','below','off','own'}
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
def clean_text(comment):
    text = str(comment)# Cleaning comments
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',' ',text)
    text = re.sub("<.*?>", " ", text)
    text = re.sub(r"[0-9]+"," ",text)
    text = re.sub(r"@[A-Za-z0-9]+"," ",text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"n\'t", ' not',text)
    text = text.replace('\\r', ' ')
    text = text.replace('\\"', ' ')
    text = text.replace('\\n', ' ')
    text = re.sub('[^A-Za-z0-9]+',' ', text)
    text = ' '.join(token for token in tokenizer.tokenize(text.lower()) if token not in stops)
    text = text.lower().strip()
    return text

df1["cleaned_comment"] = df1.swifter.apply(lambda x: clean_text(x["comment"]),axis=1)


Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

C:\Users\kapil\AppData\Local\Temp/ipykernel_14664/1636029199.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["cleaned_comment"] = df1.swifter.apply(lambda x: clean_text(x["comment"]),axis=1)


# Train-test Split

In [6]:
# Train-test Split
train_x, val_x,train_y , val_y = train_test_split(df1.drop('label',axis=1),df['label'],random_state=123,test_size=0.20)
train_txt = train_x['cleaned_comment']
val_txt = val_x['cleaned_comment']
train = pd.concat([train_x, train_y], axis=1)
val = pd.concat([val_x, val_y], axis=1)
train.drop(['comment'],axis=1,inplace=True)
val.drop(['comment'],axis=1,inplace=True)

#assigining label values as its needed for training with fasttext
train.iloc[:, 1] = train.iloc[:, 1].apply(lambda x: '__label__' + str(x))
val.iloc[:, 1] = val.iloc[:, 1].apply(lambda x: '__label__' + str(x))
train.head()

,cleaned_comment,label
472809,looks like kind ficus fig tree could not tell,__label__0
59081,donald reality tv star care tv ratings,__label__0
5664,irl,__label__1
600959,yo creia que su programa consistia en ser vene...,__label__1
961633,like one seller selling new no box,__label__1


In [7]:
#Exporting to csv file
train.to_csv(r"C:\Users\kapil\OneDrive\Desktop\NLP_Uzair\Project\Reddit_Sarcasm_Detection-main\Reddit_Sarcasm_Detection-main\src\data\raw\fasttext_train.txt")

In [8]:
#Exporting to csv file
val.to_csv(r"C:\Users\kapil\OneDrive\Desktop\NLP_Uzair\Project\Reddit_Sarcasm_Detection-main\Reddit_Sarcasm_Detection-main\src\data\raw\fasttext_val.txt")

In [9]:
#Defining export path:
fasttext_train_path = r'C:\Users\kapil\OneDrive\Desktop\NLP_Uzair\Project\Reddit_Sarcasm_Detection-main\Reddit_Sarcasm_Detection-main\src\data\raw\fasttext_train.txt'
fasttext_val_path = r'C:\Users\kapil\OneDrive\Desktop\NLP_Uzair\Project\Reddit_Sarcasm_Detection-main\Reddit_Sarcasm_Detection-main\src\data\raw\fasttext_val.txt'

#Exporting data to csv
train[['label','cleaned_comment']].to_csv(fasttext_train_path, 
                                            index = False, sep = ' ',header = None, quoting = csv.QUOTE_NONE,
                                            quotechar = "", escapechar = " ")
val[['label','cleaned_comment']].to_csv(fasttext_val_path, 
                                            index = False, sep = ' ',header = None, quoting = csv.QUOTE_NONE,
                                            quotechar = "", escapechar = " ")

In [10]:
#Defining pretrained vector path:
prerained_vec_path = r'C:\Users\kapil\OneDrive\Desktop\NLP_Uzair\Project\Reddit_Sarcasm_Detection-main\Reddit_Sarcasm_Detection-main\src\data\raw\wiki-news-300d-1M.vec'
#training model:
model = fasttext.train_supervised(fasttext_train_path, wordNgrams = 2, 
                                        epoch = 4, verbose = 1, loss = 'softmax',
                                        lr=0.099, ws = 8,dim=300,
                                        pretrainedVectors = prerained_vec_path) 

In [11]:
#Getting test results containing (dataset size, precision, recall)
model.test(fasttext_val_path)  
#(200935, 0.6887451165799886, 0.6887451165799886)

(202166, 0.6899725967769061, 0.6899725967769061)